# Mini Project: Build a Machine Learning Model

## Predict Total Fare on the NYC Taxi Dataset

Welcome to the NYC Taxi Fare Prediction project! In this Colab, we will continue using the NYC Taxi Dataset to predict the fare amount for taxi rides using a subset of available features. We will go through three main stages: building a baseline model, creating a full model, and performing hyperparameter tuning to enhance our predictions.

The previous EDA work on the NYC Taxi Dataset can be found in the [16.3.3. Student MLE MiniProject EDA](https://github.com/dmkimura/mec2-projects/blob/main/16.3.3_Student_MLE_MiniProject_EDA.ipynb) Jupyter notebook.

Now that you've completed exploratory data analysis on this dataset you should have a good understanding of the feature space.

## Project Objectives

The primary objectives of this project are as follows:

Baseline Model: We will start by building a simple baseline model to establish a benchmark for our predictions. This model will serve as a starting point to compare the performance of our subsequent models.

Full Model: Next, we will develop a more comprehensive model that leverages machine learning techniques to improve prediction accuracy. We will use Scikit-Learn's model pipeline to build a framework that enables rapid experimentation.

Hyperparameter Tuning: Lastly, we will optimize our full model by fine-tuning its hyperparameters. By systematically adjusting the parameters that control model behavior, we aim to achieve the best possible performance for our prediction task.

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

Load the NYC taxi dataset into a Pandas DataFrame and do a few basic checks to ensure the data is loaded properly. Note, there are several months of data that can be used. For simplicity, use the Yellow Taxi 2022-01 parquet file [here](https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet). Here are your tasks:

  1. Load the `yellow_tripdata_2022-01.parquet` file into Pandas.
  2. Print the first 5 rows of data.
  3. Drop any rows of data that contain NULL values.
  4. Create a new feature, 'trip_duration' that captures the duration of the trip in minutes.
  5. Create a varible named 'target_variable' to store the name of the thing we're trying to predict, 'total_amount'.
  6. Create a list called 'feature_cols' containing the feature names that we'll be using to predict our target variable. The list should contain 'VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', and 'trip_duration'.

In [66]:
# Load the dataset into a pandas DataFrame (from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
df = pd.read_parquet(
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet')

In [67]:
# Display the first few rows of the dataset
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


### Data Cleaning

Remove missing values.

In [68]:
# Drop rows with missing values.
print(f'Current missing values:\n{df.isna().sum()}')

df.dropna(inplace=True)

print(f'\nAfter removing missing values:\n{df.isna().sum()}')
#df.head()

Current missing values:
VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
dtype: int64

After removing missing values:
VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax           

### Feature engineering.
* create new feature called 'trip_duration'
* remove datetime features

**Create new feature**

The new feature will calculate the duration of each taxi ride in minutes based on the tpep_dropoff_datetime and tpep_pickup_datetime features.

In [69]:
# Create new feature, 'trip_duration'.

# from previous EDA, columns with datetime values are in UTC time
# will convert datetime values from UTC to New York time zone
df['tpep_pickup_datetime'] = df[
    'tpep_pickup_datetime'].dt.tz_localize('America/New_York', ambiguous='NaT')

df['tpep_dropoff_datetime'] = df[
    'tpep_dropoff_datetime'].dt.tz_localize('America/New_York', ambiguous='NaT')

# create variable with trip duration based on drop off and pickup times
trip_duration = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

# create new df column for trip duration expressed in minutes
df['trip_duration'] = trip_duration.dt.total_seconds() / 60

print(df.head(3))

   VendorID      tpep_pickup_datetime     tpep_dropoff_datetime  \
0         1 2022-01-01 00:35:40-05:00 2022-01-01 00:53:29-05:00   
1         1 2022-01-01 00:33:43-05:00 2022-01-01 00:42:07-05:00   
2         2 2022-01-01 00:53:21-05:00 2022-01-01 01:02:19-05:00   

   passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0              2.0           3.80         1.0                  N   
1              1.0           2.10         1.0                  N   
2              1.0           0.97         1.0                  N   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  mta_tax  \
0           142           236             1         14.5    3.0      0.5   
1           236            42             1          8.0    0.5      0.5   
2           166           166             1          7.5    0.5      0.5   

   tip_amount  tolls_amount  improvement_surcharge  total_amount  \
0        3.65           0.0                    0.3         21.95   
1        4.00    

**Remove datetime features**

Datetime datatypes are not useable datatypes for machine learning models. Since the two datetime features in the dataset were used to create a new feature, these features can be excluded from the X data used for training and testing models.

Let's create a list containing all of the features that will be used as the X dataset for model training and testing.

In [70]:
# Create a list called feature_col to store column names
target = 'total_amount'
remove_col = [target, 'tpep_dropoff_datetime', 'tpep_pickup_datetime']
feat_col = [col for col in df.columns if col not in remove_col]
feat_col

['VendorID',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'congestion_surcharge',
 'airport_fee',
 'trip_duration']

### Train, Test, Split

Splitting a dataset into training and test sets is a crucial step in machine learning model development. It allows us to evaluate the performance and generalization ability of our models accurately. The training set is used to train the model, while the test set serves as an independent sample for evaluating its performance.

1. **Model Training**: The training set is used to fit the model, allowing it to learn the underlying patterns and relationships between the features and the target variable. By exposing the model to a diverse range of examples in the training set, it can capture the underlying structure of the data.

2. **Model Evaluation**: The test set, which is independent of the training set, is crucial for evaluating how well the trained model generalizes to unseen data. It provides an unbiased assessment of the model's performance on new instances. By measuring the model's accuracy, precision, recall, or other evaluation metrics on the test set, we can estimate how well the model will perform on unseen data.

3. **Preventing Overfitting**: Overfitting occurs when a model learns the training data's noise and idiosyncrasies instead of the underlying patterns. By evaluating the model on the test set, we can identify if the model is overfitting. If the model performs significantly worse on the test set compared to the training set, it indicates overfitting. In such cases, we might need to adjust the model, feature selection, or regularization techniques to improve generalization.

4. **Hyperparameter Tuning**: Splitting the dataset allows us to perform hyperparameter tuning on the model. Hyperparameters are configuration settings that control the learning process, such as learning rate, regularization strength, or the number of hidden layers in a neural network. By using a validation set (often created from a portion of the training set), we can iteratively adjust the hyperparameters and select the best combination that maximizes the model's performance on the validation set. The final evaluation on the test set provides an unbiased estimate of the model's performance.

By splitting the dataset into training and test sets, we can ensure that our models are both well-trained and accurately evaluated. This separation helps us understand how the model will perform on new, unseen data, which is critical for assessing its effectiveness and making informed decisions about its deployment.

Here is your task:

  1. Use Scikit-Learn's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the data into training and test sets. Don't forget to set the random state.

In [71]:
# see current shape of the dataframe
df.shape

(2392428, 20)

In [72]:
# Split dataset into 70% training and 30% test sets

X = df[feat_col]
y = df[target]

X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.3, random_state=1337)

### Baseline Model

The importance of a baseline model, even if it uses a simple strategy like always predicting the mean, cannot be understated. Here's why a baseline model is valuable:

1. **Performance Comparison**: A baseline model serves as a reference point for evaluating the performance of more sophisticated models. By establishing a simple yet reasonable baseline, we can determine whether our advanced models offer any significant improvement over this basic approach. It helps us set realistic expectations and gauge the effectiveness of our efforts.

2. **Model Complexity**: A baseline model provides insight into the complexity required to solve the prediction task. If a simple strategy like predicting the median performs reasonably well, it suggests that the problem might not necessitate complex modeling techniques. Conversely, if the baseline model performs poorly, it indicates the presence of more intricate patterns that need to be captured by more sophisticated models.

3. **Minimum Performance Requirement**: A baseline model can establish a minimum performance requirement for a predictive task. If we cannot outperform the baseline, it suggests that our models have failed to capture even the most fundamental relationships within the data. In such cases, we may need to revisit our data preprocessing steps, feature engineering techniques, or consider other external factors affecting the task.

4. **Identifying Data Issues**: A baseline model can help identify potential issues within the dataset. If the baseline model performs poorly, it may indicate problems like missing values, outliers, or data inconsistencies. These issues can be further investigated and resolved to improve the overall model performance.

While a baseline model like always predicting the median may not offer the highest prediction accuracy, its importance lies in its role as a starting point for model development and evaluation. It provides a solid foundation for comparing and assessing the performance of more complex models, ensuring that any improvements made are meaningful and significant.

Here is your task:

  1. Create a model that always predicts the mean total fare of the training dataset. Use Scikit-Learn's [mean_absolute_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) to evaluate this model. Is it any good?

In [73]:
# Create a baseline model that predicts the mean of the training set
from sklearn.dummy import DummyRegressor

dum_reg = DummyRegressor(strategy='mean')

dum_reg.fit(X_train, y_train)

y_pred = dum_reg.predict(X_test)

# show that the model is predicting the mean of the
# target values in the training set
print(f'The mean value of the y_train set:\n{y_train.mean()}')
print(f'\nThe target values predicted by the baseline model:\n{y_pred}')

The mean value of the y_train set:
18.834789398226917

The target values predicted by the baseline model:
[18.8347894 18.8347894 18.8347894 ... 18.8347894 18.8347894 18.8347894]


In [74]:
# calculate the mean absolute error for the baseline model
base_mae = mean_absolute_error(y_test, y_pred)
base_mae

9.332187766854899

### Linear Regression Model

With a baseline metric in place, we can try to build a machine learning model. Obviously, if the model can't beat the baseline then there are some major issues to be resolved.

It's always a good idea to start with a simple machine learning model, like linear regression, and build upon it if necessary.

Here are your tasks:

  1. Use Scikit-Learn's [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to preprocess the categorical and continuous features independently. Apply the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to the continuous columns and [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to the categorical columns.

  One-hot encoding is a popular technique used to represent categorical variables numerically in machine learning models. It transforms categorical features into a binary vector representation, where each category is represented by a binary column. Here's an explanation of one-hot encoding:

  When working with categorical variables, such as colors (e.g., red, blue, green) or vehicle types (e.g., car, truck, motorcycle), machine learning algorithms often require numerical inputs. However, directly assigning numerical values to categories can introduce unintended relationships or orderings between them. For example, assigning the values 0, 1, and 2 to the categories red, blue, and green may imply a sequential relationship, which is not desired.

  One-hot encoding solves this problem by creating new binary columns, equal to the number of unique categories in the original feature. Each binary column represents a specific category and takes a value of 1 if the data point belongs to that category, and 0 otherwise. This encoding ensures that no implicit ordering or relationship exists between the categories.

  2. Integrate the preprocessor in the previous step with Scikit-Learn's [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model using a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

  3. Train the pipeline on the training data.

  4. Evaluate the model using mean absolute error as a metric on the test data. Does the model beat the baseline?


**Use ColumnTransformer to Preprocess Data**

In [75]:
# create list of feature names with categorial values
cat_col = list(X.select_dtypes(include=['object']).columns)
cat_col

['store_and_fwd_flag']

In [76]:
# create list of feature names with continuous values
cont_col = list(X.select_dtypes(exclude=['object']).columns)
cont_col

['VendorID',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'congestion_surcharge',
 'airport_fee',
 'trip_duration']

In [77]:
# Use Scikit-Learn's ColumnTransformer to preprocess the categorical and
# continuous features independently.
ct = ColumnTransformer(
    [("categorical", OneHotEncoder(), cat_col),
     ("continuous", StandardScaler(), cont_col)
     ]
    )

**Create pipeline to perform preprocessing and model training**

In [78]:
# Create a pipeline object containing the column transformations and regression
# model.
linr_pipe = Pipeline([('preprocessor', ct),
                    ('regressor', LinearRegression())]
            )

**Use pipeline to train the data**

In [79]:
# Fit the pipeline on the training data.
linr_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['store_and_fwd_flag']),
                                                 ('continuous',
                                                  StandardScaler(),
                                                  ['VendorID',
                                                   'passenger_count',
                                                   'trip_distance',
                                                   'RatecodeID', 'PULocationID',
                                                   'DOLocationID',
                                                   'payment_type',
                                                   'fare_amount', 'extra',
                                                   'mta_tax', 'tip_amount',
                                                   'tolls_amount',
                                                   'improvement_surcharge',
                                                   'congestion_surcharge',
                                                   'airport_fee',
                                                   'trip_duration'])])),
                ('regressor', LinearRegression())])

**Evaluate the model using mean absolute error**

In [80]:
# Make predictions on the test data.
y_linr_pred = linr_pipe.predict(X_test)

# calculate mean absolute error on the predictions
linr_mae = mean_absolute_error(y_test, y_linr_pred)

In [81]:
# display results
print(f"Baseline model's mae:\t\t{base_mae:.5f}")
print(f"Linear Regression's mae:\t{linr_mae:.5f}")

Baseline model's mae:		9.33219
Linear Regression's mae:	0.13886


The above linear regression model clearly performs better than the baseline line model. The closer the mae is to zero, the better the model performs.

### Random Forest Regression

Random Forest Regression and Linear Regression are two commonly used regression algorithms, each with its own advantages and suitability for different scenarios. Random Forest Regression offers several advantages over Linear Regression, including:

1. **Non-linearity**: Random Forest Regressor is capable of capturing non-linear relationships between features and the target variable. In contrast, Linear Regression assumes a linear relationship between the features and the target. When faced with non-linear relationships or complex feature interactions, Random Forest Regressor can provide more accurate predictions.

2. **Robustness to Outliers**: Random Forest Regressor is generally more robust to outliers compared to Linear Regression. Outliers can disproportionately impact the coefficients and predictions of Linear Regression models. However, as an ensemble of decision trees, Random Forest Regressor can mitigate the effect of outliers by averaging predictions from multiple trees.

3. **Feature Importance**: Random Forest Regressor provides a measure of feature importance, which helps identify the most influential features for making predictions. This information is useful for feature selection, understanding the underlying relationships in the data, and gaining insights into the problem domain. Unlike Linear Regression, which provides coefficient values indicating the direction and magnitude of relationships, Random Forest Regressor explicitly highlights feature importance.

4. **Handling of Categorical Variables**: Random Forest Regressor can effectively handle categorical variables without requiring pre-processing steps like one-hot encoding. It can directly incorporate categorical variables into the model, making it more convenient when working with mixed data types. In contrast, Linear Regression often requires categorical variables to be encoded or transformed before use.

5. **Handling of High-Dimensional Data**: Random Forest Regressor can handle datasets with a large number of features (high dimensionality) by automatically selecting subsets of features during the construction of individual decision trees. This reduces the risk of overfitting, which is a concern with Linear Regression when dealing with high-dimensional data.

6. **Resistance to Multicollinearity**: Random Forest Regressor is less affected by multicollinearity, which occurs when predictor variables are highly correlated. In Linear Regression, highly correlated features can lead to unstable coefficient estimates, making it challenging to interpret the individual effects of each feature. Random Forest Regressor, as an ensemble approach, is less impacted by multicollinearity because each tree is built independently.

Here are your tasks:

  1. Build a Random Forest Regressor model using Scikit-Learn's [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) and train it on the train data.

  2. Evaluate the performance of the model on the test data using mean absolute error as a metric. Mess around with various input parameter configurations to see how they affect the model. Can you beat the performance of the linear regression model?

**NOTE**

The above list of advantages for using Random Forest Regressor over Linear Regression incorrectly states that the Random Forest Regressor can take in categorical and numerical datatypes. However, the regressor simply attempts to convert all datatypes into a float. If it cannot make the conversion, such as when it tries to convert a string into a float, the regressor will fail.

However, decision trees do not require data to be standardized.

Therefore, the Random Forest Regressor model below does require that the categorial feature be converted into a numerical feature (like the Linear Regression model), but it does not require that the numerical values be standardized before training the model.

**Build and Train a Random Forest Regressor Model**

Since the data set is large, the regressor model will only create 10 trees for initial testing purposes.

In [82]:
# Build random forest regressor model
rfg1 = RandomForestRegressor(n_estimators=10)

In [83]:
# Use get.dummies() to one-hot-encode the categorical column
new_X = pd.get_dummies(X)

In [84]:
# perform train, test, split on new_X data
X1_train, X1_test, y1_train, y1_test = train_test_split(new_X, y, test_size = 0.3, random_state=1337)

In [85]:
# fit model to training data
rfg1.fit(X1_train, y1_train)

RandomForestRegressor(n_estimators=10)

**Evaluate the model using mean absolute error**

In [86]:
# Make predictions on the test data
y_rfg_pred = rfg1.predict(X1_test)

rfg1_mae = mean_absolute_error(y1_test, y_rfg_pred)

In [87]:
# display results
print(f"Baseline model's mae:\t\t\t{base_mae:.5f}")
print(f"Linear Regression's mae:\t\t{linr_mae:.5f}")
print(f"First Random Forest Regressor's mae:\t{rfg1_mae:.5f}")

Baseline model's mae:			9.33219
Linear Regression's mae:		0.13886
First Random Forest Regressor's mae:	0.02261


**Adjust Hyperparameters to See Effects on Model Performance**

*n_estimators*

In [88]:
# Build model with 20 estimators
rfg2 = RandomForestRegressor(n_estimators=20)

In [89]:
# train model
rfg2.fit(X1_train, y1_train)

RandomForestRegressor(n_estimators=20)

In [90]:
y_rfg_pred = rfg2.predict(X1_test)

rfg2_mae = mean_absolute_error(y1_test, y_rfg_pred)

*max_features*

In [91]:
# build model with 20 estimators, max_features of 10
rfg3 = RandomForestRegressor(n_estimators=20, max_features=10)

In [92]:
rfg3.fit(X1_train, y1_train)

RandomForestRegressor(max_features=10, n_estimators=20)

In [93]:
y_rfg_pred = rfg3.predict(X1_test)

rfg3_mae = mean_absolute_error(y1_test, y_rfg_pred)

*max_samples*

In [94]:
# build model with 20 estimators, max_samples size of 70%
rfg4 = RandomForestRegressor(n_estimators=20, max_samples=0.7)


In [ ]:
rfg4.fit(X1_train, y1_train)

In [ ]:
y_rfg_pred = rfg4.predict(X1_test)

rfg4_mae = mean_absolute_error(y1_test, y_rfg_pred)

*Performance of all tested models*

In [ ]:
# display results
print(f"MAE SCORES\nBaseline model:\t\t\t\t{base_mae:.5f}")
print(f"Linear Regression:\t\t\t{linr_mae:.5f}")
print(f"\nRandom Forest Regressors with:\n  10 estimators:\t\t\t{rfg1_mae:.5f}")
print(f"  20 estimators:\t\t\t{rfg2_mae:.5f}")
print(f"  20 estimators, 10 max_features:\t{rfg3_mae:.5f}")
print(f"  20 estimators, 70% max_samples:\t{rfg4_mae:.5f}")

When comparing mae scores, the Random Forest Regressor model performs the best. Simply adjusting the number of estimators from 10 to 20 provides about a 1% increase in the mae score between Random Forest models, but adjusting other features makes the model perform worse.

### Hyperparameter Tuning

Hyperparameter tuning plays a critical role in machine learning model development. It involves selecting the optimal values for the hyperparameters, which are configuration settings that control the behavior of the learning algorithm. Here's why hyperparameter tuning is so important in ML:

1. **Optimizing Model Performance**: The choice of hyperparameters can significantly impact the model's performance. By fine-tuning the hyperparameters, we can improve the model's accuracy, precision, recall, or other performance metrics. It helps to extract the maximum predictive power from the chosen algorithm and ensures that the model is well-suited to the specific problem at hand.

2. **Avoiding Overfitting and Underfitting**: Hyperparameter tuning helps strike a balance between overfitting and underfitting.

3. **Exploring Model Complexity**: Hyperparameter tuning enables us to explore the complexity of the model. For instance, in algorithms like decision trees or neural networks, we can adjust the number of layers, the number of neurons, or the maximum depth of the tree. By systematically modifying these hyperparameters, we can understand how different levels of complexity impact the model's performance and find the right balance between simplicity and complexity.

Note, there are multiple approaches to hyperparemeter tuning.  

While grid search is the easiest to understand and implement there are many advantages of Bayesian search over grid search for hyperparameter tuning:

1. **Efficiency**: Bayesian search is generally more efficient than grid search. Grid search explores all possible combinations of hyperparameter values, which can be computationally expensive and time-consuming, especially when dealing with a large number of hyperparameters or a wide range of values. Bayesian search, on the other hand, intelligently selects the next hyperparameter configuration to evaluate based on the results of previous evaluations. It focuses on areas of the hyperparameter space that are more likely to yield better performance, reducing the number of evaluations needed.

2. **Flexibility**: Bayesian search is flexible in handling continuous and discrete hyperparameters. It can handle both types of hyperparameters naturally and effectively. In contrast, grid search is more suitable for discrete hyperparameters but may struggle with continuous ones, as it requires discretization or defining a finite set of values to search over.

3. **Adaptive Search**: Bayesian search adapts its search strategy based on the results of previous evaluations. It maintains a probability distribution over the hyperparameter space, updating it with each evaluation. This allows it to dynamically allocate more evaluations to promising regions and explore unexplored areas. In contrast, grid search follows a fixed and predefined search grid, regardless of the results of previous evaluations.

4. **Better Convergence**: Bayesian search has the potential to converge to the optimal hyperparameter configuration more quickly.

Here are your tasks:

  1. Perform a grid-search on a Random Forest Regressor model. Only search the space for the parameters 'n_estimators', 'max_depth', and 'min_samples_split'. Note, this can take some time to run. Make sure you set reasonable boundaries for the search space. Use Scikit-Learn's [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) method.

  2. After you've identified the best parameters, train a random forest regression model using these parameters on the full training data.

  3. Evaluate the model from the previous step using the test data. How does your model perform?

In [ ]:
# Define the hyperparameters to tune.
param_grid = {
    'n_estimators': [5, 10, 20, 25, 50],
    'max_depth': [None, 4, 8, 12, 16],
    'min_samples_split': [2, 5, 10]
}

In [ ]:
# Perform grid search to find the best hyperparameters. This could take a while.
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best model and its parameters.
best_rfg_model = grid_search.best_estimator_
print(best_rfg_model)

#best_rfg_params = grid_search.best_params_

In [ ]:
# Fit the best classifier on the training data.
best_rfg_model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_rfg_pred = best_rfg_model.predict(X_test)

# calculate mean absolute error on the predictions
best_rfg_mae = mean_absolute_error(y_test, y_rfg_pred)


In [ ]:
# print results of all models
# display results
print(f"MAE SCORES\nBaseline model:\t\t\t\t{base_mae:.5f}")
print(f"Linear Regression:\t\t\t{linr_mae:.5f}")
print(f"\nRandom Forest Regressors with:\n  10 estimators:\t\t\t{rfg1_mae:.5f}")
print(f"  20 estimators:\t\t\t{rfg2_mae:.5f}")
print(f"  best model:\t{best_rfg_mae:.5f}")